In [1]:
import os, sys
os.sys.path.append('..')
from recommender.utils.ItemMetadata import ExplicitDataFromCSV
from recommender.utils.eval_utils import compute_auc, compute_ap
#from recommender.recommender.recommenderMF import RecommenderMF
from recommender.recommender.recommenderALS import RecommenderALS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
data_folder = 'D:\\PycharmProjects\\recommender\\data\\ml-latest-small'
ratings_csv = os.path.join(data_folder, 'ratings_sanitized.csv')
user_map_csv = os.path.join(data_folder, 'user_map.csv')
item_map_csv = os.path.join(data_folder, 'item_map.csv')
md_csv = os.path.join(data_folder, 'metadata.csv')
stats_csv = os.path.join(data_folder, 'stats.csv')
df_train = os.path.join(data_folder, 'ratings_train.csv')
df_test = os.path.join(data_folder, 'ratings_test.csv')
exp_data = ExplicitDataFromCSV(True,
                        ratings_csv=ratings_csv,
                        user_map_csv=user_map_csv,
                        item_map_csv=item_map_csv,
                        md_csv=md_csv,
                        stats_csv=stats_csv,
                        ratings_train_csv=df_train,
                        ratings_test_csv=df_test
                        )

In [4]:
model_folder = 'D:\\PycharmProjects\\recommender\\models'
model_path = os.path.join(model_folder, 'ALS_2020-02-07.22-44-24')
rmf = RecommenderALS(mode='predict', model_path=model_path)

In [5]:
# model_path = 'D:/PycharmProjects/recommender/recommender/models/model/07201914185949/'
# rmf = RecommenderMF(mode='predict', model_path=model_path)

In [6]:
F = rmf.predict(np.zeros(shape=(exp_data.M), dtype=int),np.arange(exp_data.M))

In [7]:
D = exp_data.stats.merge(exp_data.md_df, left_index=True, right_on='item_cat', how='inner')
D = D.merge(pd.DataFrame({'features': [f for f in F['q']]}), left_index=True, right_index=True)

In [8]:
D.head()

,user,title,genres,features
item_cat,,,,
0,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[0.8801038732641481, -0.4948153429947722, -0.0..."
1,110,Jumanji (1995),Adventure|Children|Fantasy,"[1.176932674209176, -0.6232892934668358, -1.06..."
2,52,Grumpier Old Men (1995),Comedy|Romance,"[0.9968068394541449, -0.284737536158248, -1.18..."
3,7,Waiting to Exhale (1995),Comedy|Drama|Romance,"[-0.8267885855747708, -0.4995954871384445, -0...."
4,49,Father of the Bride Part II (1995),Comedy,"[1.3802381587834933, -1.035058890465378, -1.90..."


In [9]:
Dviz = D[D.user>50]

In [10]:
Dviz

,user,title,genres,features
item_cat,,,,
0,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[0.8801038732641481, -0.4948153429947722, -0.0..."
1,110,Jumanji (1995),Adventure|Children|Fantasy,"[1.176932674209176, -0.6232892934668358, -1.06..."
2,52,Grumpier Old Men (1995),Comedy|Romance,"[0.9968068394541449, -0.284737536158248, -1.18..."
5,102,Heat (1995),Action|Crime|Thriller,"[0.5701320871838308, -0.10660025590028122, -0...."
6,54,Sabrina (1995),Comedy|Romance,"[1.2463545535250025, -0.8306962429927667, -0.7..."
...,...,...,...,...
8287,54,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama,"[0.2860427625891534, -0.22323284730362597, -0...."
8354,52,"Grand Budapest Hotel, The (2014)",Comedy|Drama,"[0.16839920949665943, -0.5376144913726314, -1...."
8358,73,Interstellar (2014),Sci-Fi|IMAX,"[0.5622666750655997, -0.5039904187257512, -0.7..."


In [11]:
from sklearn.manifold import TSNE
tsne = TSNE(n_iter=15000, metric="cosine")
Q = np.array([f for f in Dviz['features']])
Q_emb = tsne.fit_transform(Q)
Q_emb.shape

(436, 2)

In [14]:
import plotly
import plotly.offline as py
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [15]:
trace = go.Scatter(x=Q_emb[:,0], y=Q_emb[:,1], text=Dviz['title'], mode='markers')

In [16]:
Dviz.set_index(np.arange(len(Dviz)), inplace=True)
query_ids = np.array([f for f in Dviz.loc[Dviz.title.str.startswith('Star Wars')].index])
#query_ids = np.array([f for f in Dviz.loc[Dviz.genres.map(lambda p: 'Horror' in p.split('|'))].index])
query_emb = Q_emb[query_ids]
query_titles = Dviz.loc[query_ids]['title']
trace2 = go.Scatter(x=query_emb[:,0], y=query_emb[:,1], text=query_titles, mode='markers')
print('caculated embedding shape', Q_emb.shape)

caculated embedding shape (436, 2)


In [17]:
fig = go.Figure(data=[trace, trace2])
py.offline.iplot(fig)